In [1]:
!git clone https://github.com/Nadarsa/handwritten-ocr.git
%cd handwritten-ocr
!ls

fatal: destination path 'handwritten-ocr' already exists and is not an empty directory.
/content/handwritten-ocr
data	   notebooks  reports			  requirements.txt  src
models.md  README.md  requirements.inference.txt  results


In [2]:
!pip uninstall langchain langchain-core langchain-community -y
!pip install langchain==0.0.340
!pip install langchain-community==0.0.1

Found existing installation: langchain 0.0.340
Uninstalling langchain-0.0.340:
  Successfully uninstalled langchain-0.0.340
Found existing installation: langchain-core 0.0.13
Uninstalling langchain-core-0.0.13:
  Successfully uninstalled langchain-core-0.0.13
Found existing installation: langchain-community 0.0.1
Uninstalling langchain-community-0.0.1:
  Successfully uninstalled langchain-community-0.0.1
  Using cached langchain-0.0.340-py3-none-any.whl.metadata (16 kB)
Using cached langchain-0.0.340-py3-none-any.whl (2.0 MB)
  Using cached langchain_community-0.0.1-py3-none-any.whl.metadata (5.8 kB)
  Using cached langchain_core-0.0.13-py3-none-any.whl.metadata (978 bytes)
Using cached langchain_community-0.0.1-py3-none-any.whl (1.5 MB)
Using cached langchain_core-0.0.13-py3-none-any.whl (188 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph-prebui

In [3]:
!pip install -r requirements.inference.txt

In [4]:
import numpy as np
import pandas as pd
import sys
import time
import random
from typing import List, Tuple, Dict
from collections import defaultdict
from pathlib import Path

In [5]:
sys.path.insert(0, '/content/handwritten-ocr/src')

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from models.inference import get_ocr_model, AVAILABLE_OCR_MODELS
from data_preparation import TestDataPreparer
from metrics import MetricsCalculator, PerformanceMetrics
from evaluate import ValidationPipeline

BASE_PATH = "/content/drive/MyDrive/Практикум_3_семестр_МФТИ/handwritten-ocr"

# Параметры датасета
USE_PROCESSED = True
MAX_SAMPLES = 7           # Общее число образцов (не на каждый датасет!)
SHUFFLE = True            # Перемешать данные
SEED = 42                 # Для воспроизводимости

# Параметры валидации
MEASURE_PERFORMANCE = True    # Измерять скорость/память
SHOW_EXAMPLES = 0             # Сколько примеров показать (0 = не показывать)

# Модели для теста (None = все доступные)
MODELS_TO_TEST = 0

## Загрузка данных для тестирования

In [7]:
print(f"Базовый путь: {BASE_PATH}")
print(f"Модели: {AVAILABLE_OCR_MODELS}")

data_preparer = TestDataPreparer(base_path=BASE_PATH)

test_data = data_preparer.load_combined_dataset(
    processed=USE_PROCESSED,
    max_samples=MAX_SAMPLES,
    shuffle=SHUFFLE,
    seed=SEED
)

stats = data_preparer.get_statistics(test_data)
print(f"\nСтатистика:")
print(f"  Образцов: {stats['total_samples']}")
print(f"  Средняя длина: {stats['mean_text_length']:.1f} символов")
print(f"  Среднее слов: {stats['mean_word_count']:.1f}")

Базовый путь: /content/drive/MyDrive/Практикум_3_семестр_МФТИ/handwritten-ocr
Модели: ['trocr_kazars', 'trocr_raxtemur', 'trocr_cyrillic']
Загрузка объединённого датасета (processed)...
  HWR200: 585 образцов
  School Notebooks: 600 образцов
  Датасет перемешан (seed=42)
  Ограничено до 7 образцов
Итого: 7 образцов

Статистика:
  Образцов: 7
  Средняя длина: 1131.6 символов
  Среднее слов: 169.1


## Загрузка моделей

In [8]:
pipeline = ValidationPipeline()

models_to_test = MODELS_TO_TEST if MODELS_TO_TEST else AVAILABLE_OCR_MODELS
all_reports = []

# ЗАГРУЖАЕМ МОДЕЛИ
loaded_models = {}
for model_name in models_to_test:
    try:
        print(f"Загрузка {model_name}...")
        loaded_models[model_name] = get_ocr_model(model_name)
    except Exception as e:
        print(f"Ошибка загрузки {model_name}: {e}")

Загрузка trocr_kazars...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/273 [00:00<?, ?B/s]

Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/root/.paddlex/official_models/PP-OCRv5_server_det`.
Creating model: ('eslav_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/root/.paddlex/official_models/eslav_PP-OCRv5_mobile_rec`.


## Запуск валидационного pipeline c метриками

In [9]:
#ТЕСТИРУЕМ
for model_name, model in loaded_models.items():
    try:
        print(f"\n{'#'*60}")
        print(f"МОДЕЛЬ: {model_name}")
        print(f"{'#'*60}")

        def inference_fn(img):
            return model.predict(img)

        report = pipeline.evaluate_model(
            model_name=model_name,
            inference_fn=inference_fn,
            test_data=test_data,
            measure_performance=MEASURE_PERFORMANCE,
            show_examples=SHOW_EXAMPLES
        )

        all_reports.append(report)

    except Exception as e:
        print(f"\nОШИБКА: {e}")
        import traceback
        traceback.print_exc()


############################################################
МОДЕЛЬ: trocr_kazars
############################################################

Модель: trocr_kazars
Образцов: 7
Инференс...


Resized image size (4032x3024) exceeds max_side_limit of 4000. Resizing to fit within limit.
Resized image size (3024x4032) exceeds max_side_limit of 4000. Resizing to fit within limit.


Расчёт метрик...
Измерение производительности...


Resized image size (4032x3024) exceeds max_side_limit of 4000. Resizing to fit within limit.
Resized image size (4032x3024) exceeds max_side_limit of 4000. Resizing to fit within limit.
Resized image size (3024x4032) exceeds max_side_limit of 4000. Resizing to fit within limit.
Resized image size (4032x3024) exceeds max_side_limit of 4000. Resizing to fit within limit.



Результаты: trocr_kazars

Точность:
  CER:           54.99% (±10.45%)
  WER:           372.11% (±67.59%)
  Char Accuracy: 45.01%
  Perfect Rate:  0.00%

Производительность:
  Latency:    37414.29 ms (37.414 sec)
  P95:        54381.30 ms
  Throughput: 0.03 samples/sec
  GPU Memory: 3923.47 MB


############################################################
МОДЕЛЬ: trocr_raxtemur
############################################################

Модель: trocr_raxtemur
Образцов: 7
Инференс...


...


Результаты: trocr_raxtemur

Точность:
  CER:           42.92% (±13.25%)
  WER:           288.91% (±83.76%)
  Char Accuracy: 57.08%
  Perfect Rate:  0.00%

Производительность:
  Latency:    34206.27 ms (34.206 sec)
  P95:        50063.98 ms
  Throughput: 0.03 samples/sec
  GPU Memory: 3923.47 MB


############################################################
МОДЕЛЬ: trocr_cyrillic
############################################################

Модель: trocr_cyrillic
Образцов: 7
Инференс...


Resized image size (4032x3024) exceeds max_side_limit of 4000. Resizing to fit within limit.
Resized image size (3024x4032) exceeds max_side_limit of 4000. Resizing to fit within limit.


Расчёт метрик...
Измерение производительности...


Resized image size (4032x3024) exceeds max_side_limit of 4000. Resizing to fit within limit.
Resized image size (4032x3024) exceeds max_side_limit of 4000. Resizing to fit within limit.
Resized image size (3024x4032) exceeds max_side_limit of 4000. Resizing to fit within limit.
Resized image size (4032x3024) exceeds max_side_limit of 4000. Resizing to fit within limit.



Результаты: trocr_cyrillic

Точность:
  CER:           38.34% (±14.44%)
  WER:           259.41% (±98.62%)
  Char Accuracy: 61.66%
  Perfect Rate:  0.00%

Производительность:
  Latency:    41583.04 ms (41.583 sec)
  P95:        56061.96 ms
  Throughput: 0.02 samples/sec
  GPU Memory: 3923.47 MB



## Финальный отчет для сравнения работы моделей

In [11]:
if len(all_reports) > 1:
    print("\n" + "="*60)
    print("СРАВНЕНИЕ МОДЕЛЕЙ")
    print("="*60)

    comparison_df = pipeline.compare_models(all_reports)

    # Лучшая по точности
    best = min(all_reports, key=lambda r: r['accuracy_metrics']['mean_cer'])
    print(f"\nЛучшая точность: {best['model_name']}")
    print(f"  CER: {best['accuracy_metrics']['mean_cer']:.2f}%")


СРАВНЕНИЕ МОДЕЛЕЙ

Сравнение моделей:
         Model CER (%) WER (%) Char Acc (%) Latency (sec) Throughput
  trocr_kazars   54.99  372.11       45.01%        37.414   0.03/sec
trocr_raxtemur   42.92  288.91       57.08%        34.206   0.03/sec
trocr_cyrillic   38.34  259.41       61.66%        41.583   0.02/sec

Лучшая точность: trocr_cyrillic
  CER: 38.34%
